In [1]:
#@title
from __future__ import print_function

import os
import sys
import time
import argparse
import datetime
import math
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.autograd import Variable

#from scipy.special import softmax
#from sklearn.model_selection import train_test_split


In [2]:
#add the  bayesiancnnn and ensai src to import path - https://stackoverflow.com/a/35273613/7154530
bcnn_module_path = os.path.abspath(os.path.join('../bayesiancnn'))
if bcnn_module_path not in sys.path:
    sys.path.append(bcnn_module_path)
    
ensai_module_path = os.path.abspath(os.path.join('../ensai'))
if ensai_module_path not in sys.path:
    sys.path.append(ensai_module_path)

In [3]:
from BBBlayers import GaussianVariationalInference
from BayesianModels.BayesianAlexNet import BBBAlexNet


In [4]:
num_out = 5    #number ouf output parameters being predicted
global numpix_side
numpix_side = 192   #number of image pixels on the side
global max_noise_rms, max_psf_rms , max_cr_intensity
max_trainoise_rms = 0.1 # maximum rms of noise in training data
max_testnoise_rms = 0.1 # maximum rms of noise in test or validation data
max_noise_rms = max_testnoise_rms

max_psf_rms = 0.08/0.04  # maximum Gaussian PSF rms (in pixels)
max_cr_intensity = 0.5 # maximum scaling for cosmic ray and artefact maps

global constant_noise_rms
variable_noise_rms = True  #if True, the noise rms will be chosen randomly for each sample with a max of max_noise_rms (above)


cycle_batch_size = 50   # how many examples to read at a time (here it's equal to the batch size)
num_test_samples = 1000 # number of test samples

global pix_res
pix_res = 0.04 # pixel size in arcsec
L_side = pix_res * numpix_side

global arcs_data_path_1, arcs_data_path_2 , test_data_path_1 , test_data_path_2 , CRay_data_path
global lens_data_path_1, lens_data_path_2, testlens_data_path_1, testlens_data_path_2

global min_unmasked_flux
min_unmasked_flux = 0.75

global num_data_dirs
num_data_dirs = 3

num_training_samples = 100000
max_num_test_samples = 10000
arcs_data_path_1 = '../../data/ensai/SAURON_TEST/'
arcs_data_path_2 = arcs_data_path_1
arcs_data_path_3 = arcs_data_path_1
test_data_path_1 = '../../data/ensai/SAURON_TEST/'
test_data_path_2 = test_data_path_1
test_data_path_3 = test_data_path_1

lens_data_path_1 = '../../data/ensai/SAURON_TEST/'
lens_data_path_2 = lens_data_path_1
lens_data_path_3 = lens_data_path_1
testlens_data_path_1 = '../../data/ensai/SAURON_TEST/'
testlens_data_path_2 = testlens_data_path_1
testlens_data_path_3 = testlens_data_path_1

CRay_data_path   = '../../data/ensai/CosmicRays/'
ps_data_path = '../../data/ensai'

arcs_data_paths = [arcs_data_path_1, arcs_data_path_2, arcs_data_path_3]
test_data_paths = [test_data_path_1, test_data_path_2, test_data_path_3]
lens_data_paths = [lens_data_path_1, lens_data_path_2, lens_data_path_3]
testlens_data_paths = [testlens_data_path_1, testlens_data_path_2, testlens_data_path_3]

global max_xy_range   # xy range of center of the lens. The image is shifted in a central area with a side of max_xy_range (arcsec) during training or testing
max_xy_range = 0.5

restore_file = "../../data/ensai/trained_weights/model_9.ckpt"

In [5]:
#@title
#net_type = 'houses-bnn'
#dataset = 'houses'
net_type = 'alexnet'
dataset = 'stronglens'
outputs = num_out
inputs = 1
regression = True
resume = False
start_epoch = 0
n_epochs = 10 #set to 10 for MNIST, 30 for cifar10, and 200 for house pricing
lr = 0.001
weight_decay = 0.0005
num_samples = 10
beta_type = "Blundell"
resize=32
optim_type = 'Adam'
best_acc = 0
best_loss = np.inf
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 256
# percentage of training set to use as validation
valid_size = 0.2
#load every nth saved epoch (change to a smaller number for classification problems where num_epochs is less)
load_multiple = 1 #set to 25 (or 10) for house pricing, 1 for MNIST, 3 for cifar10
classes = []

In [6]:
# Hyper Parameter settings
use_cuda = torch.cuda.is_available()
if use_cuda is True:
    torch.cuda.set_device(0)

In [7]:
from get_data import DataProcessor

In [8]:
dataproc = DataProcessor(ps_data_path, arcs_data_paths, lens_data_paths,
                              test_data_paths, 
                              testlens_data_paths,
                              CRay_data_path, numpix_side, 
                              max_noise_rms,
                              max_psf_rms, max_cr_intensity, 
                              variable_noise_rms, pix_res, 
                              min_unmasked_flux, num_data_dirs,
                              max_xy_range, num_out)

In [15]:
def apply_transform(tensor, transform=None):
    #print(tensor.shape)
    img = TF.to_pil_image(tensor)
    img = transform(img)
    return TF.to_tensor(img)

In [16]:
transform_train = transforms.Compose([
    transforms.Resize((resize, resize))
])  

transform_test = transforms.Compose([
    transforms.Resize((resize, resize))
])

In [18]:
# read some lens image files from the disk and add observational effects to them:

max_xy_range = 0.5  # xy range of center of the lens. The image is shifted in a central area with a side of max_xy_range (arcsec) during training or testing
variable_noise_rms = True   #if True, the noise rms will be chosen randomly for each sample with a max of max_noise_rms
max_noise_rms = 0.1  # maximum rms of noise data
num_samp = 6000   #number of samples
chunk_size = batch_size    # batch number: how many test examples to pass at one time.

'''trainX = np.zeros( ( num_samp , numpix_side * numpix_side ), dtype='float32') ;   #numpy array holding the images
trainY = np.zeros( ( num_samp , num_out ) , dtype='float32' );                    #numpy array holding the lens parameters (here only used to flip for the x-y ellipticity)
trainMag = np.zeros((num_samp,1))
dataproc.read_data_batch( trainX , trainY , trainMag , num_training_samples  , 'train')             #read data
'''
X = np.zeros( ( num_samp , numpix_side * numpix_side ), dtype='float32') ;   #numpy array holding the images
Y = np.zeros( ( num_samp , num_out ) , dtype='float32' );                    #numpy array holding the lens parameters (here only used to flip for the x-y ellipticity)
mag = np.zeros((num_samp,1))
dataproc.read_data_batch( X , Y , mag , 15000  , 'test')             #read data

trainX = X[:5000, :].reshape(-1, numpix_side, numpix_side)
trainY = Y[:5000, :].reshape(-1, num_out)

testX = X[5000:, :].reshape(-1, numpix_side, numpix_side)
testY = Y[5000:, :].reshape(-1, num_out)

tensor_x_train = torch.stack([apply_transform(torch.Tensor(i), transform_train) for i in trainX]) # transform to torch tensors
tensor_y_train = torch.stack([torch.Tensor(i) for i in trainY])
trainset = torch.utils.data.TensorDataset(tensor_x_train,tensor_y_train) # create your datset

tensor_x_test = torch.stack([apply_transform(torch.Tensor(i), transform_test) for i in testX]) # transform to torch tensors
tensor_y_test = torch.stack([torch.Tensor(i) for i in testY])
testset = torch.utils.data.TensorDataset(tensor_x_test,tensor_y_test) # create your datset

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)

torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size(

torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size(

torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size(

torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size(

torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size(

...

torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size(

torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size(

torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size(

torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size(

torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size([192, 192])
torch.Size(

In [20]:
# removing image intensity bias: filter image with a 4X4 filter and remove from image
def remove_image_intensity_bias(x):
    #print(x)
    MASK = torch.abs(torch.sign(x))
    XX =  x +  ( (1-MASK) * 1000.0)
    #print(XX)
    bias_measure_filt = torch.full((1, 1, 4, 4), 1/16.0)
    bias_measure = F.conv2d(input=XX , weight=bias_measure_filt , stride=1, padding=0)
    im_bias = torch.reshape(bias_measure.min(dim=1, keepdim=True)[0].min(dim=2, keepdim=True)[0].min(dim=3, keepdim=True)[0] , [-1,1,1,1])
    return x - (im_bias * MASK )

In [21]:
net = BBBAlexNet(outputs,inputs)
if use_cuda:
    net.cuda()

loss = torch.nn.MSELoss()
vi = GaussianVariationalInference(loss, log=True)

#logfile = os.path.join('diagnostics_Bayes{}_{}.txt'.format(net_type, dataset))

In [22]:
dataloader = iter(testloader)

In [23]:
features, labels = dataloader.next()

In [24]:
f = features[0]

In [25]:
x = f.view(-1, inputs, resize, resize)

In [26]:
x_nobias = remove_image_intensity_bias(x)

In [27]:
xi = x_nobias.repeat(1, 1, 1, 1)

In [28]:
l = labels[0]

In [29]:
y = l.view(-1, outputs).repeat(1, 1)

In [30]:
vx = Variable(xi)
vy = Variable(y)

In [31]:
outs, kl = net.probforward(vx)

> /Users/pbhambhani/misc/fl/slbnn/stronglensbnns/src/bayesiancnn/BayesianModels/BayesianAlexNet.py(45)probforward()
-> for layer in self.layers:
(Pdb) c


In [32]:
outs.shape

torch.Size([1, 5])

In [33]:
outs

tensor([[ 7.0198,  6.4871,  8.3186, -5.2429,  8.7820]], grad_fn=<AddBackward0>)